# PySpark Advanced Data Manipulation - Day 2
## Advanced Transformations, Joins, and SQL Operations

Welcome to Day 2 of the PySpark workshop! Today we'll build on the fundamentals from Day 1 and explore more advanced data manipulation techniques in PySpark.

## Day 2 Agenda

Today we'll cover:
1. **Advanced DataFrame Transformations**
2. **Aggregations and Grouping**
3. **Working with SQL in Spark**
4. **Joins and Merges**
5. **Window Functions**
6. **User-Defined Functions (UDFs)**
7. **Performance Optimization Techniques**

Let's continue our PySpark journey!

## Setup and Data Loading

First, let's initialize our environment and load some data for today's exercises.

In [0]:
# Check our Spark version
print(f"Spark Version: {spark.version}")

# Create paths for our workshop data
workshop_path = "/Volumes/workspace/default/spark_workshop"
raw_data_path = f"{workshop_path}/raw_data"
processed_path = f"{workshop_path}/processed"

print("Spark environment initialized!")
print(f"Workshop path: {workshop_path}")



Spark Version: 4.0.0
Spark environment initialized!
Workshop path: /Volumes/workspace/default/spark_workshop


In [0]:
# Load the BigMart sales data from yesterday
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Create a schema for the BigMart Sales data
sales_schema = StructType([
    StructField("Item_Identifier", StringType(), False),
    StructField("Item_Weight", DoubleType(), True),
    StructField("Item_Fat_Content", StringType(), True),
    StructField("Item_Visibility", DoubleType(), True),
    StructField("Item_Type", StringType(), True),
    StructField("Item_MRP", DoubleType(), True),
    StructField("Outlet_Identifier", StringType(), False),
    StructField("Outlet_Establishment_Year", IntegerType(), True),
    StructField("Outlet_Size", StringType(), True),
    StructField("Outlet_Location_Type", StringType(), True),
    StructField("Outlet_Type", StringType(), True),
    StructField("Item_Outlet_Sales", DoubleType(), True)
])

# Read the sales data
sales_df = spark.read.format('csv')\
                  .option('header', True)\
                  .schema(sales_schema)\
                  .load(f'{workshop_path}/BigMart Sales.csv')

# Display the first few rows
display(sales_df.limit(5))

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38
NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Advanced DataFrame Transformations

Building on the basic transformations we learned yesterday, let's explore more advanced transformation techniques in PySpark.

### Data Cleaning Operations

Data cleaning is a critical part of any data processing pipeline. PySpark provides several functions to help with data cleaning tasks:

In [0]:
# Let's clean our sales data
from pyspark.sql.functions import col, when, trim, upper, regexp_replace, coalesce, lit

# Create a copy of the DataFrame for cleaning
clean_sales_df = sales_df

# 1. Standardize text fields (trim whitespace, convert to consistent case)
clean_sales_df = clean_sales_df.withColumn(
    "Item_Fat_Content", upper(trim(col("Item_Fat_Content")))
)

In [0]:
# 2. Normalize categorical values (e.g., 'low fat', 'LF', 'Low Fat' → 'LOW_FAT')
clean_sales_df = clean_sales_df.withColumn(
    "Item_Fat_Content",
    when(col("Item_Fat_Content").isin("LOW FAT", "LF"), "LOW_FAT")
    .when(col("Item_Fat_Content").isin("REG", "REGULAR"), "REGULAR")
    .otherwise(col("Item_Fat_Content"))
)

In [0]:
from pyspark.sql.functions import avg, col, coalesce, lit

# 3. Handle missing values
# - For numerical columns, replace nulls with average/median
# - For categorical columns, replace nulls with mode or a default value

# Calculate average weight by item type
avg_weight_by_type = clean_sales_df.filter(col("Item_Weight").isNotNull()) \
                        .groupBy("Item_Type") \
                        .avg("Item_Weight") \
                        .withColumnRenamed("avg(Item_Weight)", "Avg_Weight")

# Join back to original data to fill missing weights
clean_sales_df = clean_sales_df.join(
    avg_weight_by_type,
    "Item_Type",
    "left"
)

# Fill missing Item_Weight with calculated average by type
clean_sales_df = clean_sales_df.withColumn(
    "Item_Weight",
    coalesce(col("Item_Weight"), col("Avg_Weight"))
)

# Fill remaining missing weights with overall average
overall_avg_weight = clean_sales_df.filter(col("Item_Weight").isNotNull()) \
                         .select(avg(col("Item_Weight"))).collect()[0][0]

clean_sales_df = clean_sales_df.withColumn(
    "Item_Weight",
    coalesce(col("Item_Weight"), lit(overall_avg_weight))
).drop("Avg_Weight")  # Drop the temporary average column

# Fill missing Outlet_Size with 'Medium' (assuming it's the most common)
clean_sales_df = clean_sales_df.withColumn(
    "Outlet_Size",
    coalesce(col("Outlet_Size"), lit("Medium"))
)

# Display the cleaned data
display(clean_sales_df.select(
    "Item_Identifier", "Item_Weight", "Item_Fat_Content", "Outlet_Size"
).limit(10))

Item_Identifier,Item_Weight,Item_Fat_Content,Outlet_Size
FDA15,9.3,LOW_FAT,Medium
DRC01,5.92,REGULAR,Medium
FDN15,17.5,LOW_FAT,Medium
FDX07,19.2,REGULAR,Medium
NCD19,8.93,LOW_FAT,High
FDP36,10.395,REGULAR,Medium
FDO10,13.65,REGULAR,High
FDP10,12.987879554655919,LOW_FAT,Medium
FDH17,16.2,REGULAR,Medium
FDU28,19.2,REGULAR,Medium


### Column Transformations and Type Conversions

Let's explore various column transformations including type conversions, string manipulations, and date handling:

In [0]:
# Column transformations and type conversions
from pyspark.sql.functions import (
    year, month, dayofmonth, to_date, datediff, current_date,
    concat, concat_ws, substring, length, round as spark_round
)

# 1. Create a date column from the establishment year
clean_sales_df = clean_sales_df.withColumn(
    "Establishment_Date", 
    to_date(concat(col("Outlet_Establishment_Year"), lit("-01-01")), "yyyy-MM-dd")
)

In [0]:

# 2. Calculate outlet age in years (as of current date)
clean_sales_df = clean_sales_df.withColumn(
    "Outlet_Age_Years",
    year(current_date()) - col("Outlet_Establishment_Year")
)

In [0]:
# 3. Create a combined ID column using concat
clean_sales_df = clean_sales_df.withColumn(
    "Item_Outlet_ID",
    concat_ws("-", col("Item_Identifier"), col("Outlet_Identifier"))
)


In [0]:

# 4. Extract first character from Item_Identifier (indicating category)
clean_sales_df = clean_sales_df.withColumn(
    "Item_Category_Code",
    substring(col("Item_Identifier"), 1, 2)
)

# 5. Round numerical values for better presentation
clean_sales_df = clean_sales_df.withColumn(
    "Item_MRP_Rounded",
    spark_round(col("Item_MRP"), 0)
)

# 6. Create price category based on MRP
clean_sales_df = clean_sales_df.withColumn(
    "Price_Category",
    when(col("Item_MRP") < 50, "Budget")
    .when(col("Item_MRP") < 100, "Economy")
    .when(col("Item_MRP") < 200, "Standard")
    .otherwise("Premium")
)

# Display the transformed data
display(clean_sales_df.select(
    "Item_Identifier", 
    "Item_Category_Code",
    "Outlet_Identifier", 
    "Item_Outlet_ID",
    "Establishment_Date", 
    "Outlet_Age_Years",
    "Item_MRP",
    "Item_MRP_Rounded",
    "Price_Category"
).limit(10))

Item_Identifier,Item_Category_Code,Outlet_Identifier,Item_Outlet_ID,Establishment_Date,Outlet_Age_Years,Item_MRP,Item_MRP_Rounded,Price_Category
FDA15,FD,OUT049,FDA15-OUT049,1999-01-01,26,249.8092,250.0,Premium
DRC01,DR,OUT018,DRC01-OUT018,2009-01-01,16,48.2692,48.0,Budget
FDN15,FD,OUT049,FDN15-OUT049,1999-01-01,26,141.618,142.0,Standard
FDX07,FD,OUT010,FDX07-OUT010,1998-01-01,27,182.095,182.0,Standard
NCD19,NC,OUT013,NCD19-OUT013,1987-01-01,38,53.8614,54.0,Economy
FDP36,FD,OUT018,FDP36-OUT018,2009-01-01,16,51.4008,51.0,Economy
FDO10,FD,OUT013,FDO10-OUT013,1987-01-01,38,57.6588,58.0,Economy
FDP10,FD,OUT027,FDP10-OUT027,1985-01-01,40,107.7622,108.0,Standard
FDH17,FD,OUT045,FDH17-OUT045,2002-01-01,23,96.9726,97.0,Economy
FDU28,FD,OUT017,FDU28-OUT017,2007-01-01,18,187.8214,188.0,Standard


## Aggregations and Grouping

Aggregations are operations that summarize multiple rows into a single result. They are often used with `groupBy()` to perform calculations on groups of data.

Common Aggregation Functions:
- `count()`: Count rows
- `sum()`: Sum values in a column
- `avg()`: Calculate the average
- `min()`, `max()`: Find minimum and maximum values
- `first()`, `last()`: Get first or last values in a group
- `collect_list()`, `collect_set()`: Collect values into arrays

In [0]:
# Basic aggregations
from pyspark.sql.functions import count, sum, avg, min, max, first, last, collect_list, collect_set

# 1. Group by Outlet_Type and calculate sales metrics
outlet_type_metrics = clean_sales_df.groupBy("Outlet_Type").agg(
    count("*").alias("Item_Count"),
    sum("Item_Outlet_Sales").alias("Total_Sales"),
    avg("Item_Outlet_Sales").alias("Avg_Sales_Per_Item"),
    min("Item_Outlet_Sales").alias("Min_Sales"),
    max("Item_Outlet_Sales").alias("Max_Sales")
)

display(outlet_type_metrics)

# 2. Group by Item_Type and calculate sales and price metrics
item_type_metrics = clean_sales_df.groupBy("Item_Type").agg(
    count("*").alias("Item_Count"),
    sum("Item_Outlet_Sales").alias("Total_Sales"),
    avg("Item_MRP").alias("Avg_Price"),
    collect_set("Outlet_Identifier").alias("Available_In_Outlets")
)

display(item_type_metrics)

# 3. Multi-level grouping: Outlet_Type and Item_Type
multi_level_metrics = clean_sales_df.groupBy("Outlet_Type", "Item_Type").agg(
    count("*").alias("Item_Count"),
    sum("Item_Outlet_Sales").alias("Total_Sales"),
    avg("Item_Outlet_Sales").alias("Avg_Sales")
).orderBy("Outlet_Type", "Total_Sales", ascending=[True, False])

display(multi_level_metrics)

Outlet_Type,Item_Count,Total_Sales,Avg_Sales_Per_Item,Min_Sales,Max_Sales
Supermarket Type3,935,3453926.0514,3694.038557647059,241.6854,13086.9648
Supermarket Type1,5577,1.2917342262999993E7,2316.1811481083005,73.238,10256.649
Grocery Store,1083,368034.266,339.8285004616805,33.29,1775.6886
Supermarket Type2,928,1851822.8300000012,1995.4987392241392,69.2432,6768.5228


Item_Type,Item_Count,Total_Sales,Avg_Price,Available_In_Outlets
Canned,649,1444151.4926,139.76383204930661,"List(OUT018, OUT046, OUT045, OUT035, OUT049, OUT019, OUT013, OUT017, OUT010, OUT027)"
Meat,425,917565.6119999998,139.88203247058826,"List(OUT035, OUT027, OUT049, OUT010, OUT013, OUT018, OUT045, OUT019, OUT046, OUT017)"
Starchy Foods,148,351401.2504,147.83802297297296,"List(OUT046, OUT027, OUT049, OUT017, OUT018, OUT010, OUT045, OUT013, OUT035, OUT019)"
Baking Goods,648,1265525.3421999998,126.38076604938273,"List(OUT010, OUT013, OUT019, OUT017, OUT046, OUT018, OUT035, OUT027, OUT045, OUT049)"
Snack Foods,1200,2732786.0870000003,146.19493366666669,"List(OUT013, OUT035, OUT049, OUT046, OUT010, OUT027, OUT045, OUT018, OUT017, OUT019)"
Breakfast,110,232298.95160000003,141.78815090909092,"List(OUT046, OUT013, OUT049, OUT035, OUT019, OUT010, OUT017, OUT045, OUT018, OUT027)"
Dairy,682,1522594.0511999996,148.4992076246334,"List(OUT017, OUT027, OUT013, OUT035, OUT045, OUT019, OUT018, OUT010, OUT049, OUT046)"
Household,910,2055493.7132,149.42475318681318,"List(OUT018, OUT019, OUT010, OUT027, OUT017, OUT045, OUT049, OUT013, OUT046, OUT035)"
Seafood,64,148868.2194,141.84171875,"List(OUT035, OUT027, OUT045, OUT049, OUT013, OUT046, OUT010, OUT019, OUT018, OUT017)"
Frozen Foods,856,1825734.7885999999,138.50336612149533,"List(OUT045, OUT010, OUT035, OUT013, OUT018, OUT027, OUT019, OUT049, OUT017, OUT046)"


Outlet_Type,Item_Type,Item_Count,Total_Sales,Avg_Sales
Grocery Store,Snack Foods,146,51596.171,353.3984315068493
Grocery Store,Fruits and Vegetables,152,48602.068400000004,319.75045
Grocery Store,Household,119,43707.106799999994,367.2866117647058
Grocery Store,Frozen Foods,103,33195.456399999995,322.2859844660194
Grocery Store,Dairy,92,31451.7262,341.8665891304348
Grocery Store,Canned,73,25759.136199999997,352.86487945205477
Grocery Store,Meat,66,25576.707000000006,387.52586363636374
Grocery Store,Baking Goods,85,24827.0162,292.08254352941174
Grocery Store,Health and Hygiene,67,24515.4218,365.9018179104478
Grocery Store,Soft Drinks,54,18554.514400000004,343.6021185185186


### Pivot Operations

Pivot operations transform rows into columns, creating a cross-tabulation of data. This is useful for reporting and analysis purposes:

In [0]:
# Pivot example: Item types across different outlet types
pivot_df = clean_sales_df.groupBy("Item_Type").pivot(
    "Outlet_Type"
).agg(
    sum("Item_Outlet_Sales").alias("Total_Sales")
).orderBy("Item_Type")

display(pivot_df)

# Fill nulls with zeros for better presentation
from pyspark.sql.functions import coalesce, lit

# Get list of all Outlet_Type columns (they become column names after pivot)
outlet_type_cols = [col for col in pivot_df.columns if col != "Item_Type"]

# Replace nulls with zeros
for col_name in outlet_type_cols:
    pivot_df = pivot_df.withColumn(col_name, coalesce(col(col_name), lit(0)))

display(pivot_df)

# Another pivot example: Average price by Item_Type across Price_Category
price_pivot = clean_sales_df.groupBy("Item_Type").pivot(
    "Price_Category"
).agg(
    count("Item_Identifier").alias("Item_Count")  # FIXED: use a specific column
).orderBy("Item_Type")

display(price_pivot)

Item_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Baking Goods,24827.0162,895461.0519999998,121065.07719999997,224172.19680000003
Breads,12604.925599999997,375508.5368,54604.9212,110518.80519999997
Breakfast,7843.7898000000005,151526.75879999998,22496.0504,50432.3526
Canned,25759.136199999997,1000560.2451999997,145681.70059999995,272150.41059999994
Dairy,31451.7262,1076694.4752000002,152130.63940000004,262317.2104
Frozen Foods,33195.456399999995,1292668.0107999989,183599.01059999998,316272.31080000004
Fruits and Vegetables,48602.068400000004,1931957.8521999991,263471.70759999997,576028.1886000003
Hard Drinks,5835.737,323554.1653999999,49445.636999999995,78957.8878
Health and Hygiene,24515.4218,712754.879200001,108845.64980000003,199084.18699999998
Household,43707.106799999994,1437219.8487999996,196267.18719999996,378299.5704000001


Item_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Baking Goods,24827.0162,895461.0519999998,121065.07719999997,224172.19680000003
Breads,12604.925599999997,375508.5368,54604.9212,110518.80519999997
Breakfast,7843.7898000000005,151526.75879999998,22496.0504,50432.3526
Canned,25759.136199999997,1000560.2451999997,145681.70059999995,272150.41059999994
Dairy,31451.7262,1076694.4752000002,152130.63940000004,262317.2104
Frozen Foods,33195.456399999995,1292668.0107999989,183599.01059999998,316272.31080000004
Fruits and Vegetables,48602.068400000004,1931957.8521999991,263471.70759999997,576028.1886000003
Hard Drinks,5835.737,323554.1653999999,49445.636999999995,78957.8878
Health and Hygiene,24515.4218,712754.879200001,108845.64980000003,199084.18699999998
Household,43707.106799999994,1437219.8487999996,196267.18719999996,378299.5704000001


Item_Type,Budget,Economy,Premium,Standard
Baking Goods,66,199,64,319
Breads,23,54,53,121
Breakfast,9,20,15,66
Canned,29,180,111,329
Dairy,57,169,181,275
Frozen Foods,95,188,166,407
Fruits and Vegetables,93,220,217,702
Hard Drinks,31,21,25,137
Health and Hygiene,68,90,59,303
Household,82,106,177,545


## Working with SQL in Spark

PySpark allows you to execute SQL queries on your DataFrames by registering them as temporary views. This is powerful for users familiar with SQL and enables complex data manipulations using SQL syntax.

In [0]:
# Register our cleaned sales DataFrame as a temporary view
clean_sales_df.createOrReplaceTempView("sales")

# Now we can use SQL to query the data
sales_by_outlet_sql = spark.sql("""
SELECT 
    Outlet_Type,
    COUNT(*) AS Item_Count,
    SUM(Item_Outlet_Sales) AS Total_Sales,
    AVG(Item_Outlet_Sales) AS Avg_Sales_Per_Item,
    MIN(Item_Outlet_Sales) AS Min_Sales,
    MAX(Item_Outlet_Sales) AS Max_Sales
FROM sales
GROUP BY Outlet_Type
ORDER BY Total_Sales DESC
""")

display(sales_by_outlet_sql)

# Complex SQL query with multiple aggregations and conditions
sales_analysis_sql = spark.sql("""
SELECT 
    Item_Type,
    COUNT(*) AS Item_Count,
    SUM(Item_Outlet_Sales) AS Total_Sales,
    AVG(Item_MRP) AS Avg_Price,
    AVG(CASE WHEN Item_Fat_Content = 'LOW_FAT' THEN Item_Outlet_Sales ELSE 0 END) AS Avg_LowFat_Sales,
    AVG(CASE WHEN Item_Fat_Content = 'REGULAR' THEN Item_Outlet_Sales ELSE 0 END) AS Avg_Regular_Sales,
    SUM(CASE WHEN Price_Category = 'Premium' THEN Item_Outlet_Sales ELSE 0 END) / SUM(Item_Outlet_Sales) * 100 AS Premium_Sales_Percentage
FROM sales
WHERE Item_Outlet_Sales > 0
GROUP BY Item_Type
HAVING COUNT(*) > 10
ORDER BY Total_Sales DESC
""")

display(sales_analysis_sql)

Outlet_Type,Item_Count,Total_Sales,Avg_Sales_Per_Item,Min_Sales,Max_Sales
Supermarket Type1,5577,1.2917342262999993E7,2316.1811481083005,73.238,10256.649
Supermarket Type3,935,3453926.0514,3694.038557647059,241.6854,13086.9648
Supermarket Type2,928,1851822.8300000012,1995.4987392241392,69.2432,6768.5228
Grocery Store,1083,368034.266,339.8285004616805,33.29,1775.6886


Item_Type,Item_Count,Total_Sales,Avg_Price,Avg_LowFat_Sales,Avg_Regular_Sales,Premium_Sales_Percentage
Fruits and Vegetables,1232,2820059.8168000015,144.58123457792206,1116.0029353896095,1173.0066561688332,29.688643581682456
Snack Foods,1200,2732786.087000002,146.19493366666669,1390.907799499999,886.4139396666664,28.380308026648677
Household,910,2055493.7131999983,149.42475318681318,2258.7843002197783,0.0,29.510691378163283
Frozen Foods,856,1825734.7885999978,138.50336612149533,1113.1903768691584,1019.6773668224295,35.62696947341284
Dairy,682,1522594.0511999999,148.49920762463336,1314.7841568914944,917.7584401759528,43.14537287744264
Canned,649,1444151.4925999993,139.76383204930647,1160.5109436055463,1064.6839602465334,29.038835949612903
Baking Goods,648,1265525.3422,126.38076604938273,895.8698614197526,1057.1013456790117,18.56626747525593
Health and Hygiene,520,1045200.1378000001,130.81892076923077,2010.0002650000004,0.0,20.666105675670334
Meat,425,917565.6119999996,139.88203247058814,814.2467679999997,1344.7311425882344,23.213606745323418
Soft Drinks,445,892897.7219999991,131.49250561797746,1710.6945505617975,295.8171842696629,16.512687440813085


## Joins and Merges

Joining DataFrames is a common operation in data processing. PySpark supports various types of joins similar to SQL:

- **Inner Join**: Only matching records from both DataFrames
- **Left Join**: All records from left DataFrame, matching from right
- **Right Join**: All records from right DataFrame, matching from left
- **Full Outer Join**: All records from both DataFrames
- **Cross Join**: Cartesian product of both DataFrames
- **Semi Join**: Filter left DataFrame for rows matching the right DataFrame
- **Anti Join**: Filter left DataFrame for rows not matching the right DataFrame

Let's create a new DataFrame to join with our sales data:

In [0]:
# Create a simple item_categories DataFrame
from pyspark.sql import Row

item_categories_data = [
    Row(Category_Code="FD", Category_Name="Food", Perishable=True),
    Row(Category_Code="DR", Category_Name="Drinks", Perishable=True),
    Row(Category_Code="NC", Category_Name="Non-Consumable", Perishable=False),
    Row(Category_Code="HH", Category_Name="Household", Perishable=False)
]

item_categories_df = spark.createDataFrame(item_categories_data)
display(item_categories_df)

# Extract item category code from Item_Identifier for joining
# We'll create a more complete version of the earlier Item_Category_Code
sales_with_category = clean_sales_df.withColumn(
    "Category_Code",
    substring(col("Item_Identifier"), 1, 2)
)

# Display a few rows to check the Category_Code
display(sales_with_category.select("Item_Identifier", "Category_Code").limit(5))

Category_Code,Category_Name,Perishable
FD,Food,true
DR,Drinks,true
NC,Non-Consumable,false
HH,Household,false


Item_Identifier,Category_Code
FDA15,FD
DRC01,DR
FDN15,FD
FDX07,FD
NCD19,NC


In [0]:
# Now let's perform various types of joins

# 1. Inner Join: Only keep items with matching category codes
inner_join_df = sales_with_category.join(
    item_categories_df,
    sales_with_category.Category_Code == item_categories_df.Category_Code,
    "inner"
)

print("Inner Join Count:", inner_join_df.count())
display(inner_join_df.select(
    sales_with_category.Item_Identifier, 
    sales_with_category.Category_Code,  # Disambiguate
    item_categories_df.Category_Name, 
    item_categories_df.Perishable, 
    sales_with_category.Item_Type, 
    sales_with_category.Item_MRP
).limit(5))

# 2. Left Join: Keep all sales records, add category info when available
left_join_df = sales_with_category.join(
    item_categories_df,
    sales_with_category.Category_Code == item_categories_df.Category_Code,
    "left"
)

print("Left Join Count:", left_join_df.count())
display(left_join_df.select(
    sales_with_category.Item_Identifier, 
    sales_with_category.Category_Code,  # Disambiguate
    item_categories_df.Category_Name, 
    item_categories_df.Perishable, 
    sales_with_category.Item_Type, 
    sales_with_category.Item_MRP
).limit(5))

# 3. Right Join: Keep all categories, with sales data when available
right_join_df = sales_with_category.join(
    item_categories_df,
    sales_with_category.Category_Code == item_categories_df.Category_Code,
    "right"
)

print("Right Join Count:", right_join_df.count())
display(right_join_df.select(
    sales_with_category.Item_Identifier, 
    sales_with_category.Category_Code,  # Disambiguate
    item_categories_df.Category_Name, 
    item_categories_df.Perishable, 
    sales_with_category.Item_Type, 
    sales_with_category.Item_MRP
).limit(5))

# 4. Full Outer Join: Keep everything from both DataFrames
full_join_df = sales_with_category.join(
    item_categories_df,
    sales_with_category.Category_Code == item_categories_df.Category_Code,
    "full"
)

print("Full Join Count:", full_join_df.count())
display(full_join_df.select(
    sales_with_category.Item_Identifier, 
    sales_with_category.Category_Code,  # Disambiguate
    item_categories_df.Category_Name, 
    item_categories_df.Perishable, 
    sales_with_category.Item_Type, 
    sales_with_category.Item_MRP
).limit(5))

Inner Join Count: 8523


Item_Identifier,Category_Code,Category_Name,Perishable,Item_Type,Item_MRP
FDA15,FD,Food,true,Dairy,249.8092
DRC01,DR,Drinks,true,Soft Drinks,48.2692
FDN15,FD,Food,true,Meat,141.618
FDX07,FD,Food,true,Fruits and Vegetables,182.095
NCD19,NC,Non-Consumable,false,Household,53.8614


Left Join Count: 8523


Item_Identifier,Category_Code,Category_Name,Perishable,Item_Type,Item_MRP
FDA15,FD,Food,true,Dairy,249.8092
DRC01,DR,Drinks,true,Soft Drinks,48.2692
FDN15,FD,Food,true,Meat,141.618
FDX07,FD,Food,true,Fruits and Vegetables,182.095
NCD19,NC,Non-Consumable,false,Household,53.8614


Right Join Count: 8524


Item_Identifier,Category_Code,Category_Name,Perishable,Item_Type,Item_MRP
NCD19,NC,Non-Consumable,false,Household,53.8614
NCB42,NC,Non-Consumable,false,Health and Hygiene,115.3492
NCB30,NC,Non-Consumable,false,Household,196.5084
NCD06,NC,Non-Consumable,false,Household,45.906
NCS17,NC,Non-Consumable,false,Health and Hygiene,96.4436


Full Join Count: 8524


Item_Identifier,Category_Code,Category_Name,Perishable,Item_Type,Item_MRP
NCD19,NC,Non-Consumable,false,Household,53.8614
NCB42,NC,Non-Consumable,false,Health and Hygiene,115.3492
NCB30,NC,Non-Consumable,false,Household,196.5084
NCD06,NC,Non-Consumable,false,Household,45.906
NCS17,NC,Non-Consumable,false,Health and Hygiene,96.4436


### Self Joins and Multi-way Joins

PySpark also supports self-joins (joining a DataFrame with itself) and joining multiple DataFrames together:

In [0]:
# Let's create some additional data for multi-way joins
# Create a simple outlet information DataFrame
outlet_data = [
    Row(Outlet_Identifier="OUT010", Region="North", Company="RetailCorp"),
    Row(Outlet_Identifier="OUT013", Region="South", Company="SuperMart"),
    Row(Outlet_Identifier="OUT017", Region="East", Company="RetailCorp"),
    Row(Outlet_Identifier="OUT018", Region="West", Company="SuperMart"),
    Row(Outlet_Identifier="OUT019", Region="North", Company="MegaStore"),
    Row(Outlet_Identifier="OUT027", Region="South", Company="MegaStore"),
    Row(Outlet_Identifier="OUT035", Region="West", Company="RetailCorp"),
    Row(Outlet_Identifier="OUT045", Region="East", Company="SuperMart"),
    Row(Outlet_Identifier="OUT046", Region="North", Company="MegaStore"),
    Row(Outlet_Identifier="OUT049", Region="South", Company="RetailCorp")
]

outlet_df = spark.createDataFrame(outlet_data)
display(outlet_df)

# Multi-way join: Sales data with both category and outlet information
multi_join_df = sales_with_category.join(
    item_categories_df,
    sales_with_category.Category_Code == item_categories_df.Category_Code,
    "left"
).join(
    outlet_df,
    sales_with_category.Outlet_Identifier == outlet_df.Outlet_Identifier,
    "left"
).select(
    sales_with_category.Item_Identifier,
    sales_with_category.Item_Type,
    item_categories_df.Category_Name,
    sales_with_category.Item_MRP,
    sales_with_category.Outlet_Identifier,
    outlet_df.Region,
    outlet_df.Company,
    sales_with_category.Item_Outlet_Sales
)

display(multi_join_df.limit(10))

# Analyze sales by region and category
region_category_sales = multi_join_df.groupBy("Region", "Category_Name").agg(
    sum("Item_Outlet_Sales").alias("Total_Sales"),
    avg("Item_Outlet_Sales").alias("Avg_Sales"),
    count("*").alias("Item_Count")
).orderBy("Region", "Total_Sales", ascending=[True, False])

display(region_category_sales)

Outlet_Identifier,Region,Company
OUT010,North,RetailCorp
OUT013,South,SuperMart
OUT017,East,RetailCorp
OUT018,West,SuperMart
OUT019,North,MegaStore
OUT027,South,MegaStore
OUT035,West,RetailCorp
OUT045,East,SuperMart
OUT046,North,MegaStore
OUT049,South,RetailCorp


Item_Identifier,Item_Type,Category_Name,Item_MRP,Outlet_Identifier,Region,Company,Item_Outlet_Sales
FDA15,Dairy,Food,249.8092,OUT049,South,RetailCorp,3735.138
DRC01,Soft Drinks,Drinks,48.2692,OUT018,West,SuperMart,443.4228
FDN15,Meat,Food,141.618,OUT049,South,RetailCorp,2097.27
FDX07,Fruits and Vegetables,Food,182.095,OUT010,North,RetailCorp,732.38
NCD19,Household,Non-Consumable,53.8614,OUT013,South,SuperMart,994.7052
FDP36,Baking Goods,Food,51.4008,OUT018,West,SuperMart,556.6088
FDO10,Snack Foods,Food,57.6588,OUT013,South,SuperMart,343.5528
FDP10,Snack Foods,Food,107.7622,OUT027,South,MegaStore,4022.7636
FDH17,Frozen Foods,Food,96.9726,OUT045,East,SuperMart,1076.5986
FDU28,Frozen Foods,Food,187.8214,OUT017,East,RetailCorp,4710.535


Region,Category_Name,Total_Sales,Avg_Sales,Item_Count
East,Food,3048054.371399999,2296.951297211755,1327
East,Non-Consumable,751891.2690000006,2173.0961531791922,346
East,Drinks,404245.1305999999,2221.1270912087907,182
North,Food,1821015.5981999985,1270.7715270062795,1433
North,Non-Consumable,470251.2109999999,1193.5309923857865,394
North,Drinks,195162.62499999997,1049.2614247311826,186
South,Food,5696233.923399992,2819.9177838613823,2020
South,Non-Consumable,1435752.4255999995,2771.7228293436283,518
South,Drinks,648573.0908,2504.143207722008,259
West,Food,3003740.7208000016,2233.264476431228,1345


## Window Functions

Window functions perform calculations across a set of rows related to the current row. They're similar to group by aggregations but don't collapse the rows.

Common use cases for window functions:
- Calculating running totals or moving averages
- Ranking items within groups
- Computing percentiles
- Accessing values from preceding or following rows

In [0]:
# Window function examples
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    rank, dense_rank, row_number, lag, lead, 
    ntile, percent_rank, cume_dist
)

# 1. Basic window: Rank items by sales within each outlet type
window_by_outlet_type = Window.partitionBy("Outlet_Type").orderBy(col("Item_Outlet_Sales").desc())

ranked_sales = sales_with_category.withColumn(
    "Sales_Rank", rank().over(window_by_outlet_type)
).withColumn(
    "Sales_Dense_Rank", dense_rank().over(window_by_outlet_type)
).withColumn(
    "Sales_Row_Number", row_number().over(window_by_outlet_type)
)

# Display top 3 items by sales for each outlet type
display(ranked_sales.filter(col("Sales_Rank") <= 3).select(
    "Outlet_Type", 
    "Item_Identifier", 
    "Item_Type", 
    "Item_Outlet_Sales", 
    "Sales_Rank", 
    "Sales_Dense_Rank", 
    "Sales_Row_Number"
).orderBy("Outlet_Type", "Sales_Rank"))

Outlet_Type,Item_Identifier,Item_Type,Item_Outlet_Sales,Sales_Rank,Sales_Dense_Rank,Sales_Row_Number
Grocery Store,NCK30,Household,1775.6886,1,1,1
Grocery Store,FDJ55,Meat,1575.2828,2,2,2
Grocery Store,FDV59,Breads,1524.0162,3,3,3
Supermarket Type1,FDF39,Dairy,10256.649,1,1,1
Supermarket Type1,NCM05,Health and Hygiene,9779.9362,2,2,2
Supermarket Type1,FDK03,Dairy,9664.7528,3,3,3
Supermarket Type2,FDC02,Canned,6768.5228,1,1,1
Supermarket Type2,FDU51,Meat,6729.9064,2,2,2
Supermarket Type2,FDY51,Meat,6611.394,3,3,3
Supermarket Type3,NCE42,Household,13086.9648,1,1,1


In [0]:


# 2. Running totals and percentages
window_by_outlet_ordered = Window.partitionBy("Outlet_Type").orderBy("Item_Outlet_Sales").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

running_totals = sales_with_category.withColumn(
    "Running_Sales_Total", sum("Item_Outlet_Sales").over(window_by_outlet_ordered)
)

# Get total sales per outlet type for calculating percentages
outlet_totals = sales_with_category.groupBy("Outlet_Type").agg(
    sum("Item_Outlet_Sales").alias("Outlet_Total_Sales")
)

# Join to calculate running percentage
running_totals = running_totals.join(
    outlet_totals,
    "Outlet_Type",
    "left"
).withColumn(
    "Running_Sales_Percentage", 
    col("Running_Sales_Total") / col("Outlet_Total_Sales") * 100
)

display(running_totals.select(
    "Outlet_Type", 
    "Item_Identifier", 
    "Item_Outlet_Sales", 
    "Running_Sales_Total", 
    "Outlet_Total_Sales", 
    "Running_Sales_Percentage"
).orderBy("Outlet_Type", "Item_Outlet_Sales").limit(10))

Outlet_Type,Item_Identifier,Item_Outlet_Sales,Running_Sales_Total,Outlet_Total_Sales,Running_Sales_Percentage
Grocery Store,DRK12,33.29,33.29,368034.266,0.009045353401957414
Grocery Store,NCR42,33.29,66.58,368034.266,0.01809070680391483
Grocery Store,FDU21,33.9558,100.5358,368034.266,0.02731696727391139
Grocery Store,NCL41,34.6216,135.1574,368034.266,0.0367241348119471
Grocery Store,FDQ47,35.2874,170.4448,368034.266,0.04631220941802196
Grocery Store,FDS40,36.619,207.0638,368034.266,0.05626209816017512
Grocery Store,FDS40,36.619,243.6828,368034.266,0.06621198690232827
Grocery Store,FDQ36,37.2848,280.9676,368034.266,0.07634278271252058
Grocery Store,FDH28,37.9506,318.9182,368034.266,0.08665448559075203
Grocery Store,DRK35,37.9506,356.8688,368034.266,0.09696618846898349


In [0]:


# 3. Comparing with previous/next rows using lag/lead
window_by_type_ordered = Window.partitionBy("Item_Type").orderBy("Item_MRP")

comparison_df = sales_with_category.withColumn(
    "Prev_Item_MRP", lag("Item_MRP", 1).over(window_by_type_ordered)
).withColumn(
    "Next_Item_MRP", lead("Item_MRP", 1).over(window_by_type_ordered)
).withColumn(
    "Price_Diff_From_Prev", col("Item_MRP") - col("Prev_Item_MRP")
).withColumn(
    "Price_Diff_To_Next", col("Next_Item_MRP") - col("Item_MRP")
)

display(comparison_df.select(
    "Item_Type", 
    "Item_Identifier", 
    "Item_MRP", 
    "Prev_Item_MRP", 
    "Next_Item_MRP", 
    "Price_Diff_From_Prev", 
    "Price_Diff_To_Next"
).orderBy("Item_Type", "Item_MRP").limit(15))

Item_Type,Item_Identifier,Item_MRP,Prev_Item_MRP,Next_Item_MRP,Price_Diff_From_Prev,Price_Diff_To_Next
Baking Goods,FDT36,33.9874,null,35.0874,null,1.1000000000000014
Baking Goods,FDT36,35.0874,33.9874,35.0874,1.1000000000000014,0.0
Baking Goods,FDP12,35.0874,35.0874,35.2874,0.0,0.19999999999999574
Baking Goods,FDT36,35.2874,35.0874,35.6848,0.19999999999999574,0.39740000000000464
Baking Goods,FDQ36,35.6848,35.2874,35.6874,0.39740000000000464,0.0025999999999939405
Baking Goods,FDT36,35.6874,35.6848,35.9874,0.0025999999999939405,0.30000000000000426
Baking Goods,FDP12,35.9874,35.6874,36.1874,0.30000000000000426,0.19999999999999574
Baking Goods,FDP12,36.1874,35.9874,36.2848,0.19999999999999574,0.09740000000000038
Baking Goods,FDQ36,36.2848,36.1874,36.7164,0.09740000000000038,0.4316000000000031
Baking Goods,FDW23,36.7164,36.2848,36.8874,0.4316000000000031,0.17099999999999937


## User-Defined Functions (UDFs)

User-Defined Functions (UDFs) allow you to define custom functions in Python that can be applied to DataFrame columns. UDFs are useful when built-in functions don't meet your specific needs.

Note: UDFs can impact performance as they require serialization/deserialization between the JVM and Python. For performance-critical operations, consider using built-in functions or Pandas UDFs.

In [0]:
# Example of creating and using UDFs
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType, BooleanType

# 1. Simple UDF to categorize items based on price
def categorize_price(price):
    if price is None:
        return "Unknown"
    elif price < 50:
        return "Budget"
    elif price < 100:
        return "Economy"
    elif price < 200:
        return "Standard"
    else:
        return "Premium"

# Register the UDF with Spark
price_category_udf = udf(categorize_price, StringType())

# Apply the UDF to our DataFrame
udf_result = sales_with_category.withColumn(
    "Price_Category_UDF", 
    price_category_udf(col("Item_MRP"))
)

display(udf_result.select(
    "Item_Identifier", 
    "Item_MRP", 
    "Price_Category_UDF"
).limit(10))

# 2. UDF to calculate a profit metric (simplified)
def calculate_profit(mrp, sales):
    if mrp is None or sales is None:
        return None
    # Assume a simple relationship between MRP and profit
    return sales - (mrp * 0.7)

profit_udf = udf(calculate_profit, DoubleType())

# Apply the UDF
udf_result = udf_result.withColumn(
    "Estimated_Profit", 
    profit_udf(col("Item_MRP"), col("Item_Outlet_Sales"))
)

display(udf_result.select(
    "Item_Identifier", 
    "Item_MRP", 
    "Item_Outlet_Sales", 
    "Estimated_Profit"
).limit(10))

# 3. UDF with multiple column inputs
def is_high_value_item(item_type, price, sales):
    if None in (item_type, price, sales):
        return False
    
    # Different thresholds for different item types
    if item_type == "Fruits and Vegetables" and price > 100 and sales > 2000:
        return True
    elif item_type == "Dairy" and price > 150 and sales > 1500:
        return True
    elif price > 200 and sales > 2500:
        return True
    else:
        return False

high_value_udf = udf(is_high_value_item, BooleanType())

# Apply the UDF
udf_result = udf_result.withColumn(
    "Is_High_Value", 
    high_value_udf(col("Item_Type"), col("Item_MRP"), col("Item_Outlet_Sales"))
)

# Show high value items
display(udf_result.filter(col("Is_High_Value") == True).select(
    "Item_Identifier", 
    "Item_Type", 
    "Item_MRP", 
    "Item_Outlet_Sales", 
    "Is_High_Value"
).orderBy("Item_Type", "Item_MRP"))

Item_Identifier,Item_MRP,Price_Category_UDF
FDA15,249.8092,Premium
DRC01,48.2692,Budget
FDN15,141.618,Standard
FDX07,182.095,Standard
NCD19,53.8614,Economy
FDP36,51.4008,Economy
FDO10,57.6588,Economy
FDP10,107.7622,Standard
FDH17,96.9726,Economy
FDU28,187.8214,Standard


Item_Identifier,Item_MRP,Item_Outlet_Sales,Estimated_Profit
FDA15,249.8092,3735.138,3560.27156
DRC01,48.2692,443.4228,409.63436
FDN15,141.618,2097.27,1998.1374
FDX07,182.095,732.38,604.9135
NCD19,53.8614,994.7052,957.00222
FDP36,51.4008,556.6088,520.62824
FDO10,57.6588,343.5528,303.19164
FDP10,107.7622,4022.7636,3947.3300600000002
FDH17,96.9726,1076.5986,1008.7177800000001
FDU28,187.8214,4710.535,4579.06002


Item_Identifier,Item_Type,Item_MRP,Item_Outlet_Sales,Is_High_Value
FDJ12,Baking Goods,206.8296,4154.592,true
FDJ12,Baking Goods,208.8296,5400.9696,true
FDO48,Baking Goods,219.7456,3094.6384,true
FDO48,Baking Goods,219.8456,4863.0032,true
FDA48,Baking Goods,221.4114,5099.3622,true
FDA48,Baking Goods,222.0114,3547.3824,true
FDX36,Baking Goods,223.9404,4950.8888,true
FDX36,Baking Goods,224.0404,5851.0504,true
FDT12,Baking Goods,224.5062,4514.124,true
FDX36,Baking Goods,224.8404,3600.6464,true


## Performance Optimization Techniques

Performance optimization is crucial when working with large datasets. PySpark provides several techniques to improve query performance:

1. **Caching/Persistence**: Store intermediate results in memory
2. **Partitioning**: Control how data is distributed
3. **Broadcast joins**: Optimize small table joins
4. **Coalesce/Repartition**: Manage the number of partitions
5. **Bucketing**: Pre-organize data by certain columns

In [0]:
# Performance optimization examples

# 1. Caching/Persistence
# Cache a DataFrame that will be used multiple times
sales_with_category.cache()  # Store in memory
# or use persist() for more control
# sales_with_category.persist(StorageLevel.MEMORY_AND_DISK)

# Check if the DataFrame is cached
print("Is cached:", spark.catalog.isCached("sales"))


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7731533994687345>, line 5
      1 # Performance optimization examples
      2 
      3 # 1. Caching/Persistence
      4 # Cache a DataFrame that will be used multiple times
----> 5 sales_with_category.cache()  # Store in memory
      6 # or use persist() for more control
      7 # sales_with_category.persist(StorageLevel.MEMORY_AND_DISK)
      8 
      9 # Check if the DataFrame is cached
     10 print("Is cached:", spark.catalog.isCached("sales"))

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2163, in DataFrame.cache(self)
   2162 def cache(self) -> "DataFrame":
-> 2163     return self.persist()

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2172, in DataFrame.persist(self, storageLevel)
   2167 def persist(
   2168     self,
   2169     stor

In [0]:


# 2. Broadcast joins for small tables
from pyspark.sql.functions import broadcast

# Broadcasting the smaller DataFrame (item_categories_df) improves join performance
broadcast_join = sales_with_category.join(
    broadcast(item_categories_df),
    sales_with_category.Category_Code == item_categories_df.Category_Code,
    "left"
)

In [0]:
# 3. Repartitioning to control parallelism
# Reduce the number of partitions (use when you have too many small partitions)
repartitioned_df = sales_with_category.coalesce(4)

# Increase the number of partitions (use when you have too few large partitions)
# repartitioned_df = sales_with_category.repartition(8)

# In Databricks serverless, RDD APIs are not supported, so you cannot use .rdd.getNumPartitions().
# To inspect partitioning, use DataFrame transformations or mapInPandas/mapInArrow if needed.

# Use mapInPandas to count the number of partitions
import pandas as pd
from pyspark.sql.types import IntegerType, StructType, StructField

def partition_counter(iterator):
    for i, pdf in enumerate(iterator):
        yield pd.DataFrame({'partition_id': [i]})

partition_schema = StructType([StructField('partition_id', IntegerType(), False)])
partition_ids = repartitioned_df.mapInPandas(partition_counter, schema=partition_schema)
num_partitions = partition_ids.select('partition_id').distinct().count()
print(f"Number of partitions (via mapInPandas): {num_partitions}")

Number of partitions (via mapInPandas): 1


In [0]:
# 4. Partition by specific columns (use when filtering/joining on these columns)
repartitioned_by_col = sales_with_category.repartition("Outlet_Type")



In [0]:
# 5. Advanced: Partitioning and Z-Ordering (Delta Lake optimization)
# Bucketing is NOT supported for Delta tables. Use partitionBy or Z-Ordering instead.
# Create a partitioned Delta table for repeated joins or queries on the same column
sales_with_category.write.partitionBy(
    "Outlet_Type"  # Partition by Outlet_Type
).format("delta").mode("overwrite").saveAsTable(
    "partitioned_sales"
)

# For further optimization, you can use Z-ORDER after writing the Delta table:
# spark.sql("OPTIMIZE partitioned_sales ZORDER BY (Outlet_Type)")

# When you're done with cached data, uncache it to free up memory
# sales_with_category.unpersist()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-7731533994687355>, line 4
      1 # 5. Advanced: Partitioning and Z-Ordering (Delta Lake optimization)
      2 # Bucketing is NOT supported for Delta tables. Use partitionBy or Z-Ordering instead.
      3 # Create a partitioned Delta table for repeated joins or queries on the same column
----> 4 sales_with_category.write.partitionBy(
      5     "Outlet_Type"  # Partition by Outlet_Type
      6 ).format("delta").mode("overwrite").saveAsTable(
      7     "partitioned_sales"
      8 )

NameError: name 'sales_with_category' is not defined

## Summary of Day 2

Today we've covered advanced data manipulation techniques in PySpark:

1. **Advanced DataFrame Transformations**: Data cleaning, column transformations, and type conversions
2. **Aggregations and Grouping**: Summarizing data with groupBy, agg, and pivot operations
3. **SQL in Spark**: Using SQL syntax for complex data manipulations
4. **Joins and Merges**: Combining data from multiple sources with different join types
5. **Window Functions**: Performing calculations across related rows without collapsing them
6. **User-Defined Functions**: Creating custom functions for specialized transformations
7. **Performance Optimization**: Techniques to improve query efficiency and speed

In Day 3, we'll dive into:
- Delta Lake for reliable data lakes
- Data validation and quality checks
- Machine learning with MLlib
- Structured streaming for real-time data processing

Feel free to experiment with the techniques we've learned today!

## Day 2 Practice Exercises

Let's solidify your understanding of advanced PySpark concepts with these practice exercises:

### Exercise 1: Advanced Data Cleaning and Transformation
1. Identify and standardize different variations of values in a categorical column (e.g., find and standardize different ways "Low Fat" is represented in the "Item_Fat_Content" column).
2. Create a date column from an integer year and calculate the age of each record relative to the current date.
3. Implement a data quality check that flags rows with outlier values in the "Item_MRP" column (values more than 2 standard deviations from the mean).
4. Write a utility function that takes a DataFrame and replaces null values in each column with appropriate defaults (mean for numeric columns, mode for categorical columns).

### Exercise 2: Advanced Aggregations
1. Calculate the total sales, average sales, and count of items for each combination of "Item_Type" and "Outlet_Type".
2. Find the top 3 selling items (by "Item_Outlet_Sales") in each outlet.
3. Use pivot to create a cross-tabulation of "Item_Type" by "Outlet_Type" showing the average price of items.
4. Calculate the running total of sales when sorted by "Item_MRP" within each "Item_Type".

### Exercise 3: SQL Operations and Joins
1. Create temporary views for both the sales data and outlet data.
2. Write a SQL query that joins these views and calculates the percentage of total sales contributed by each outlet type.
3. Implement the same query using DataFrame operations (no SQL).
4. Create a self-join on the sales data to find pairs of items that have the same Item_Type but different Outlet_Types.

### Exercise 4: Window Functions and UDFs
1. Use window functions to rank items by their sales within each outlet.
2. Calculate the difference in MRP between each item and the next highest priced item in the same category.
3. Create a UDF that calculates a "value score" based on a formula that combines "Item_MRP" and "Item_Outlet_Sales".
4. Apply this UDF to create a new column and then find the top 5 items by value score.

### Exercise 5: Performance Optimization
1. Cache a filtered subset of the sales data and measure the execution time of a complex query before and after caching.
2. Create a bucketed table from the sales data, with bucketing on "Item_Type".
3. Compare the performance of a join operation with and without broadcasting.
4. Examine the execution plan of a query and identify potential optimization opportunities.

### Bonus Challenge
Implement a comprehensive data processing pipeline that:
1. Ingests the raw sales data
2. Cleans and validates the data using custom functions
3. Performs multiple aggregations and transformations
4. Joins with reference data
5. Outputs multiple analysis results to different formats (Parquet, Delta, CSV)
6. Includes proper error handling and logging